# Data Cleaning- Telegram Analysis

In [2]:
#import relevant libraries
import pandas as pd
import numpy as np
import datetime
import regex as re

In [3]:
#read in the data
telegram_df = pd.read_csv('final_channel_dataframe.csv')

/var/folders/fj/lzb0vlnn46524bnhg4b1wvl80000gn/T/ipykernel_58525/535943241.py:3: DtypeWarning: Columns (21,25,27,29,32) have mixed types. Specify dtype option on import or set low_memory=False.
  telegramdf2 = pd.read_csv('missing_channel_dataframe.csv')


In [1]:
#look at data
#telegram_df.head(5)

In [4]:
#Check for percentage missing values 
telegram_df.isnull().sum()/len(telegram_df)

Unnamed: 0            0.000000
_                     0.000000
id                    0.000000
peer_id               0.000000
date                  0.000000
message               0.185087
out                   0.000000
mentioned             0.000000
media_unread          0.000000
silent                0.000000
post                  0.000000
from_scheduled        0.002112
legacy                0.000000
edit_hide             0.002112
pinned                0.002112
noforwards            0.002112
from_id               1.000000
fwd_from              0.926924
via_bot_id            1.000000
reply_to              0.902684
media                 0.314481
reply_markup          0.996108
entities              0.002112
views                 0.002140
forwards              0.002140
replies               0.834762
edit_date             0.271114
post_author           0.993163
grouped_id            0.745100
reactions             0.354842
restriction_reason    0.002112
ttl_period            1.000000
action  

In [8]:
#Message has some null values -- if it is null it means it was simply a media file, so we should drop those rows. 
#dropping all rows with missing values for message
telegram_df = telegram_df.dropna(subset=['message'], how='all')

In [6]:
#Check again for percentage missing values 
telegram_df.isnull().sum()/len(telegram_df)

Unnamed: 0            0.000000
_                     0.000000
id                    0.000000
peer_id               0.000000
date                  0.000000
message               0.000000
out                   0.000000
mentioned             0.000000
media_unread          0.000000
silent                0.000000
post                  0.000000
from_scheduled        0.000000
legacy                0.000000
edit_hide             0.000000
pinned                0.000000
noforwards            0.000000
from_id               1.000000
fwd_from              0.934153
via_bot_id            1.000000
reply_to              0.905239
media                 0.383316
reply_markup          0.995268
entities              0.000000
views                 0.000035
forwards              0.000035
replies               0.803158
edit_date             0.171794
post_author           0.992263
grouped_id            0.907675
reactions             0.221874
restriction_reason    0.000000
ttl_period            1.000000
action  

In [9]:
#Need to drop columns that have virtually 100 percent missing values or over 70% missing vals: 
#from_id, fwd_from, via_bot_id, reply_to, reply_markup, post_author, action, ttl_period, grouped_id
#post author will be null because its from a channel not a specific author 
telegram_df = telegram_df.drop(columns = ['from_id', 'fwd_from', 'via_bot_id', 'reply_to', 'reply_markup', 'replies',
                           'post_author', 'action', 'ttl_period', 'grouped_id', 'Unnamed: 0', '_'])

In [8]:
#checking again now that I removed those high na columns
telegram_df.isnull().sum()/len(telegram_df)

id                    0.000000
peer_id               0.000000
date                  0.000000
message               0.000000
out                   0.000000
mentioned             0.000000
media_unread          0.000000
silent                0.000000
post                  0.000000
from_scheduled        0.000000
legacy                0.000000
edit_hide             0.000000
pinned                0.000000
noforwards            0.000000
media                 0.383316
entities              0.000000
views                 0.000035
forwards              0.000035
edit_date             0.171794
reactions             0.221874
restriction_reason    0.000000
dtype: float64

In [9]:
#Drop columns that don't tell us any relevant information
#entities doesn't tell you anything important in this context - drop
#noforwards column: binary telling you whether or not there were no forwards - unnecesary since we 
#already have a count of forwards - drop
#'out' column not providing any useful information - all false 
#same thing with mentioned column - all false 
#media_unread - also all false
#post - all true - they are all post from a channel so this will always be true 
#legacy is also all false
#Need to drop media column bc that is the actual media embedded in the post, not useful for NLP purposes
#need to drop media because it is unusable in this context
telegram_df = telegram_df.drop(columns = ['entities','noforwards', 'out', 'mentioned', 
                                          'media_unread', 'post', 'legacy', 'media'])
telegram_df.head()

,id,peer_id,date,message,silent,from_scheduled,edit_hide,pinned,views,forwards,edit_date,reactions,restriction_reason
0,69180,"{'_': 'PeerChannel', 'channel_id': 1199360700}",2023-02-11 22:24:28+00:00,Реал Мадрид – клубный чемпион мира.\n\nУкраинс...,True,0.0,1.0,0.0,1405220.0,752.0,2023-02-11T22:24:42+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]
1,69179,"{'_': 'PeerChannel', 'channel_id': 1199360700}",2023-02-11 21:37:52+00:00,«Було три влучання ворожих ракет С-300 у місті...,True,0.0,1.0,0.0,1391782.0,933.0,2023-02-11T21:38:01+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]
2,69178,"{'_': 'PeerChannel', 'channel_id': 1199360700}",2023-02-11 21:09:25+00:00,Вибухи у Харкові.\n\nОкупанти завдають ударів ...,True,0.0,0.0,0.0,1385758.0,944.0,2023-02-11T21:13:41+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]
3,69177,"{'_': 'PeerChannel', 'channel_id': 1199360700}",2023-02-11 20:53:12+00:00,Сегодня над территорией США заметили еще 3 воз...,True,0.0,0.0,0.0,1385580.0,2186.0,2023-02-11T20:55:43+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]
4,69176,"{'_': 'PeerChannel', 'channel_id': 1199360700}",2023-02-11 20:11:56+00:00,"Штурм сэконда в Киевском Ирпене.\n\nВот, где с...",False,0.0,1.0,0.0,1360378.0,9469.0,2023-02-11T20:12:02+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]


In [10]:
#Change id to message id so its more clear
telegram_df = telegram_df.rename({'id': 'message_id'}, axis = 1)
telegram_df.head()

,message_id,peer_id,date,message,silent,from_scheduled,edit_hide,pinned,views,forwards,edit_date,reactions,restriction_reason
0,69180,"{'_': 'PeerChannel', 'channel_id': 1199360700}",2023-02-11 22:24:28+00:00,Реал Мадрид – клубный чемпион мира.\n\nУкраинс...,True,0.0,1.0,0.0,1405220.0,752.0,2023-02-11T22:24:42+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]
1,69179,"{'_': 'PeerChannel', 'channel_id': 1199360700}",2023-02-11 21:37:52+00:00,«Було три влучання ворожих ракет С-300 у місті...,True,0.0,1.0,0.0,1391782.0,933.0,2023-02-11T21:38:01+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]
2,69178,"{'_': 'PeerChannel', 'channel_id': 1199360700}",2023-02-11 21:09:25+00:00,Вибухи у Харкові.\n\nОкупанти завдають ударів ...,True,0.0,0.0,0.0,1385758.0,944.0,2023-02-11T21:13:41+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]
3,69177,"{'_': 'PeerChannel', 'channel_id': 1199360700}",2023-02-11 20:53:12+00:00,Сегодня над территорией США заметили еще 3 воз...,True,0.0,0.0,0.0,1385580.0,2186.0,2023-02-11T20:55:43+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]
4,69176,"{'_': 'PeerChannel', 'channel_id': 1199360700}",2023-02-11 20:11:56+00:00,"Штурм сэконда в Киевском Ирпене.\n\nВот, где с...",False,0.0,1.0,0.0,1360378.0,9469.0,2023-02-11T20:12:02+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]


In [11]:
#peer_id is the channel id
#change name of column to channel id to make more clear 
telegram_df = telegram_df.rename({'peer_id': 'channel_id'}, axis = 1)

In [12]:
#Getting the channel id's
#extract just the last numbers of the actual channel id 
telegram_df['channel_id'] = telegram_df['channel_id'].str.split("'channel_id':").str[-1]
telegram_df['channel_id'] = telegram_df['channel_id'].str.split("}").str[0]

#check to see it worked
telegram_df.head()

,message_id,channel_id,date,message,silent,from_scheduled,edit_hide,pinned,views,forwards,edit_date,reactions,restriction_reason
0,69180,1199360700,2023-02-11 22:24:28+00:00,Реал Мадрид – клубный чемпион мира.\n\nУкраинс...,True,0.0,1.0,0.0,1405220.0,752.0,2023-02-11T22:24:42+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]
1,69179,1199360700,2023-02-11 21:37:52+00:00,«Було три влучання ворожих ракет С-300 у місті...,True,0.0,1.0,0.0,1391782.0,933.0,2023-02-11T21:38:01+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]
2,69178,1199360700,2023-02-11 21:09:25+00:00,Вибухи у Харкові.\n\nОкупанти завдають ударів ...,True,0.0,0.0,0.0,1385758.0,944.0,2023-02-11T21:13:41+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]
3,69177,1199360700,2023-02-11 20:53:12+00:00,Сегодня над территорией США заметили еще 3 воз...,True,0.0,0.0,0.0,1385580.0,2186.0,2023-02-11T20:55:43+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]
4,69176,1199360700,2023-02-11 20:11:56+00:00,"Штурм сэконда в Киевском Ирпене.\n\nВот, где с...",False,0.0,1.0,0.0,1360378.0,9469.0,2023-02-11T20:12:02+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[]


In [246]:
#Now I need to create a new column that is the channel name
#create a dictionary so that I can have a column with the channel name as well as the channel_id 

#for i in telegram_df['channel_id'].unique(): 
#    print(i)
channel_names = {1199360700: 'Труха⚡️Украина', 
           1478765631: 'Мир сегодня с "Юрий Подоляка', 
           1197363285: 'Украина Сейчас: новости, война, Россия', 
           1198589840: 'Реальная Война | Украина Новости', 
           1352726486: 'Инсайдер UA', 
           1233777422: 'Украина Online', 
           1307866449: 'Всевидящее ОКО 🇺🇦 Украина',
           1536630827: 'Лачен пише', 
           1172830166: 'Телеграмна служба новин - Україна',
           1400569744: 'Top News | (Війна, Україна, Новини)', 
           1297540669: 'Український Телеграм 🇺🇦', 
           1343028414: 'Анатолий Шарий', 
           1463721328: 'Zelenskiy / Official',
            1305722586: 'ТСН новини / ТСН.ua',
            1181169156: 'Реальный Киев | Новости Украина 🇺🇦', 
           1242259852: 'Легитимный', 
           1438194508: 'Боже, яке кончене!🇦', 
           1271765197: 'Резидент', 
           1280273449:'Ukraine NOW', 
           1242446516: 'Украина 24/7', 
           1206439755: 'Київ INFO | Новости Украина 🇺🇦', 
           1747601201: 'Віталій Кім / Миколаївська ОДА', 
           1498303038: 'Хуевая Одесса', 
           1505028797: 'Карта повітряних тривог 🇺🇦', 
           1450213869: 'Одесса INFO ⚓️ 🇺🇦 Новости', 
            1708761316: 'Прямой Эфир', 
            1141171940: 'Kadyrov_95', 
            1432477212: 'СМИ Россия не Москва', 
            1101170442: 'РИА Новости', 
            1124946559: 'Леонардо Дайвинчик', 
            1117628569: 'Mash', 
            1260622817: 'Readovka', 
            1202206821: 'ФИЛЬМЫ КИНО СЕРИАЛЫ', 
            1394050290: 'Раньше всех. Ну почти.', 
            1254661214: 'Осторожно, новости', 
            1315735637: 'СОЛОВЬЁВ', 
            1621525678: 'Дима Масленников Блоггер', 
            1135021433: 'WarGonzo', 
            1003313758: 'Новости Москвы', 
            1497011710: 'Кровавая барыня', 
            1563378267: 'Медуза — LIVE', 
            1288489154: 'ТОПОР - Горячие новости', 
            1355540894: 'Операция Z: Военкоры Русской Весны'
           }

#making the ids an integer so I can maap the dictionary
telegram_df['channel_id'] = telegram_df['channel_id'].astype('int')

In [247]:
#Map the dictionary with the column to create the channel names
telegram_df['channel_name'] = telegram_df['channel_id'].map(channel_names)
telegram_df.sample(7)

,message_id,channel_id,date,message,silent,from_scheduled,edit_hide,pinned,views,forwards,edit_date,reactions,restriction_reason,channel_name
251774,13899,1536630827,2022-06-17 16:58:06+00:00,Британські компанії хочуть допомогти відновити...,False,0.0,1.0,0.0,315193.0,168.0,2022-06-17T16:58:10+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[],Лачен пише
714229,42743,1260622817,2022-09-28 08:50:38+00:00,Сенатор РФ Джабаров возмутился обвинением Росс...,False,0.0,1.0,0.0,432571.0,223.0,2023-02-02T04:24:33+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[],Readovka
637851,2945,1141171940,2022-10-07 21:12:18+00:00,В Грозном масштабно отмечают 70-летие нашего д...,True,0.0,1.0,0.0,1149636.0,636.0,2022-12-23T16:31:20+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[],Kadyrov_95
177920,26756,1352726486,2022-03-18 19:36:09+00:00,Макрон в телефонном разговоре с Путиным призва...,True,0.0,1.0,0.0,285738.0,138.0,2022-03-18T19:36:17+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[],Инсайдер UA
432809,20077,1181169156,2022-04-25 07:41:15+00:00,⚡️Итоги визита Госсекретаря США Энтони Блинкен...,True,0.0,0.0,0.0,213441.0,142.0,2022-04-25T07:41:26+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[],Реальный Киев | Новости Украина 🇺🇦
780845,149610,1315735637,2023-01-05 09:19:13+00:00,⚡️⚡️⚡️⚡️ Путин и Эрдоган провели телефонный ра...,False,0.0,1.0,0.0,390800.0,69.0,2023-01-05T09:19:25+00:00,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[],СОЛОВЬЁВ
844196,51960,1003313758,2023-01-02 07:07:23+00:00,🎅 Люди пользуются возможностью носить костюмы ...,True,0.0,1.0,0.0,587482.0,1117.0,2023-01-02T07:07:27+00:00,NaN,[],Новости Москвы


In [248]:
#DEALING WITH REACTIONS NOW 
#looking at the possible reactions
telegram_df['reactions'].unique()

#checking the structure of 1 row 
telegram_df['reactions'][0]

"{'_': 'MessageReactions', 'results': [{'_': 'ReactionCount', 'reaction': {'_': 'ReactionEmoji', 'emoticon': '❤'}, 'count': 13062, 'chosen_order': None}, {'_': 'ReactionCount', 'reaction': {'_': 'ReactionEmoji', 'emoticon': '👍'}, 'count': 3136, 'chosen_order': None}, {'_': 'ReactionCount', 'reaction': {'_': 'ReactionEmoji', 'emoticon': '😁'}, 'count': 623, 'chosen_order': None}, {'_': 'ReactionCount', 'reaction': {'_': 'ReactionEmoji', 'emoticon': '🔥'}, 'count': 442, 'chosen_order': None}, {'_': 'ReactionCount', 'reaction': {'_': 'ReactionEmoji', 'emoticon': '🤬'}, 'count': 329, 'chosen_order': None}, {'_': 'ReactionCount', 'reaction': {'_': 'ReactionEmoji', 'emoticon': '😢'}, 'count': 75, 'chosen_order': None}], 'min': False, 'can_see_list': False, 'recent_reactions': []}"

In [249]:
telegram_df['reactions'] = telegram_df['reactions'].fillna("0")
def extract_emoji_counts(reaction):
    pattern = r"'emoticon':\s+'(.)'.+?'count':\s+(\d+)"
    emoji_counts = {}
    for i in reaction:
        matches = re.findall(pattern, reaction)
        for emoji, count in matches:
            emoji_counts[emoji] = count
    return emoji_counts

#reactions_list = telegram_df['reactions'][0:12]
# apply the function to each row of the dataframe
emoji_counts_df = telegram_df['reactions'].apply(extract_emoji_counts)


In [285]:
df = emoji_counts_df.to_frame()
df.to_csv('emoji_version1.csv')

In [254]:
df1 = pd.read_csv('emoji_version1.csv')
df1.head()

,Unnamed: 0,reactions
0,0,"{'❤': '13062', '👍': '3136', '😁': '623', '🔥': '..."
1,1,"{'🤬': '16199', '👍': '2834', '😢': '2000', '😁': ..."
2,2,"{'🤬': '18662', '👍': '4055', '😢': '1404', '🔥': ..."
3,3,"{'😁': '11313', '🤬': '6912', '👍': '2446', '😢': ..."
4,4,"{'😁': '15730', '🤬': '8854', '😢': '2735', '👍': ..."


In [286]:
# extract all unique emojis used across all tweets
all_emojis = set()
for emoji_dict in df['reactions']:
    all_emojis.update(emoji_dict.keys())

# create new columns for each unique emoji
for emoji in all_emojis:
    df[emoji] = 0

In [287]:
# define a custom function to populate the new columns with emoji counts
def populate_counts(row):
    for emoji, count in row['reactions'].items():
        row[emoji] = count
    return row

# apply the custom function to each row of the data frame
df = df.apply(populate_counts, axis=1)

# drop the original 'reactions' column
df = df.drop('reactions', axis=1)

In [300]:
non_zero_cols = (df != 0).any(axis=0)

# select only the columns where this condition is True
df2 = df.loc[:, non_zero_cols]
#check the columns are the same length as the set of unique emojis 
print(len(df2.columns) == len(all_emojis))

df2.columns

True


Index(['😐', '👏', '🎉', '💋', '🤣', '👌', '😱', '🎅', '🥰', '😘', '🤝', '👎', '👀', '🙏',
       '💯', '🖕', '🆒', '❤', '😴', '😈', '🤩', '👻', '⚡', '💩', '🥱', '😁', '👍', '🗿',
       '💔', '🕊', '🙉', '🐳', '🔥', '😍', '💘', '🤪', '🙈', '🤗', '🤯', '🌚', '😇', '🤨',
       '🤮', '🙊', '☃', '🎄', '🏆', '🍌', '😢', '💅', '🌭', '🤔', '🥴', '🎃', '🍾', '🤓',
       '🤡', '😭', '✍', '🦄', '💊', '🍓', '🤬', '😨'],
      dtype='object')

In [306]:
#add the two dataframes together and save to a csv
result_df = pd.concat([telegram_df, df2], axis=1)
result_df = result_df.drop(columns = 'restriction_reason')

In [ ]:
#create a list of the emoji columns and then sum up across them to create a total_reactions column
total_emoji_cols = ['😐', '👏', '🎉', '💋', '🤣', '👌', '😱', '🎅', '🥰', '😘', '🤝', '👎', '👀', '🙏',
       '💯', '🖕', '🆒', '❤', '😴', '😈', '🤩', '👻', '⚡', '💩', '🥱', '😁', '👍', '🗿',
       '💔', '🕊', '🙉', '🐳', '🔥', '😍', '💘', '🤪', '🙈', '🤗', '🤯', '🌚', '😇', '🤨',
       '🤮', '🙊', '☃', '🎄', '🏆', '🍌', '😢', '💅', '🌭', '🤔', '🥴', '🎃', '🍾', '🤓',
       '🤡', '😭', '✍', '🦄', '💊', '🍓', '🤬', '😨']

#turning them to int in case they werent alredy
result_df[total_emoji_cols] = result_df[total_emoji_cols].astype(int)

result_df['total_reactions'] = result_df[total_emoji_cols].sum(axis=1)

In [ ]:
#add in crude ideology as a column 
#first define the channels and their ideologies
ideology_mapping = {
    "pro-russia": ['Мир сегодня с "Юрий Подоляка"', 'Анатолий Шарий',  'Легитимный', 'Резидент', 'Прямой Эфир' ,
                  'Kadyrov_95', 'СМИ Россия не Москва', 'РИА Новости', 'Readovka', 'Правдивости', 'РОССИЯ СЕЙЧАС', 
                  'VOBLYA', 'СОЛОВЬЁВ', 'WarGonzo', 'Операция Z: Военкоры Русской Весны'],
    
    "pro-ukraine": ['Труха⚡️Украина', 'Украина Сейчас: новости, война, Россия', 'Реальная Война | Украина Новости', 'Инсайдер UA', 
                   'Украина Online', 'Всевидящее ОКО 🇺🇦 Украина', 'Лачен пише', 'Телеграмна служба новин - Україна', 
                   'Top News | (Війна, Україна, Новини)', 'Український Телеграм 🇺🇦', 'Zelenskiy / Official', 
                    'ТСН новини / ТСН.ua', 'Реальный Киев | Новости Украина 🇺🇦', 'Ukraine NOW', 'Ukraine NOW', 
                   'Київ INFO | Новости Украина 🇺🇦', 'Віталій Кім / Миколаївська ОДА', 
                    'Карта повітряних тривог 🇺🇦', 'Одесса INFO ⚓️ 🇺🇦 Новости'],
    
    "independent": ['Раньше всех. Ну почти.', 'Осторожно, новости', 'Кровавая барыня', 'Медуза — LIVE', 'ТОПОР - Горячие новости'],
    "entertainment": ['Хуевая Одесса', 'Леонардо Дайвинчик', 'Свежак', 'Наука и Факты', 'Mash', 'ФИЛЬМЫ КИНО СЕРИАЛЫ', 
                     'ЛИТВИН', 'Новости Москвы', 'Дима Масленников Блоггер'],
    "anti-russian": 'Боже, яке кончене!'
}

def map_ideology(channel_name):
    for ideology, keywords in ideology_mapping.items():
        if any(keyword in channel_name for keyword in keywords):
            return ideology
    return "NA"

# Apply the function to create the 'ideology' column
result_df['ideology'] = result_df['channel_name'].apply(map_ideology)


In [ ]:
#set index to false so you dont have an extra index row when you read data in later on
result_df.to_csv("cleaned_telegram.csv", index = False)

In [308]:
#double check final file
results = pd.read_csv('cleaned_telegram.csv')
results.head()

,Unnamed: 0,message_id,channel_id,date,message,silent,from_scheduled,edit_hide,pinned,views,...,🍾,🤓,🤡,😭,✍,🦄,💊,🍓,🤬,😨
0,0,69180,1199360700,2023-02-11 22:24:28+00:00,Реал Мадрид – клубный чемпион мира.\n\nУкраинс...,True,0.0,1.0,0.0,1405220.0,...,0,0,0,0,0,0,0,0,329,0
1,1,69179,1199360700,2023-02-11 21:37:52+00:00,«Було три влучання ворожих ракет С-300 у місті...,True,0.0,1.0,0.0,1391782.0,...,0,0,0,0,0,0,0,0,16199,0
2,2,69178,1199360700,2023-02-11 21:09:25+00:00,Вибухи у Харкові.\n\nОкупанти завдають ударів ...,True,0.0,0.0,0.0,1385758.0,...,0,0,0,0,0,0,0,0,18662,0
3,3,69177,1199360700,2023-02-11 20:53:12+00:00,Сегодня над территорией США заметили еще 3 воз...,True,0.0,0.0,0.0,1385580.0,...,0,0,0,0,0,0,0,0,6912,0
4,4,69176,1199360700,2023-02-11 20:11:56+00:00,"Штурм сэконда в Киевском Ирпене.\n\nВот, где с...",False,0.0,1.0,0.0,1360378.0,...,0,0,0,0,0,0,0,0,8854,0
